# Generate plots
Auxiliar codes to generate optical signal samples plots as well as comparision between signals form different distances or real vs. modified.

An example function to expand/translate a constellation point is also included to exemplify the comparison, but the data modification should be performed using the attack methodologies proposed.

In [1]:
# Libraries for correct code execution 

import os, time
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.mixture import GaussianMixture

import cv2

In [2]:
# Chunk to mount Google Drive on Google Colab 

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# List with the users that will execute the code.
workers = ["Ronald", "Local"]

# Modify index to change the user.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

In [4]:
# Parameters + paths.

input_data_path=path+"/rawData"
distances=[1,15]
nsamples=50
span_length=80
nsymbols=2048

######
output_data_path=path+"/trainingData"
min_dist=0
max_dist=3000
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
trainingProp=0.8
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]
######

In [5]:
X=None
Y=[]
colnames=['i'+str(i) for i in range(nsymbols)]

for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)

In [6]:
def strToTuple(s):
    s_aux=s.split("i")
    s=s_aux[0]+"j"
    return complex(s)
X=X.applymap(strToTuple)

In [7]:
obs_d_80 = X.iloc[2,].to_list()
obs_d_1200 = X.iloc[55,].to_list()

In [8]:
def extract_coords(df):
  # extract real part
  x = [ele.real for ele in df]
  # extract imaginary part
  y = [ele.imag for ele in df]
  return x,y

In [9]:
def generate_plot(df):
  # extract real part
  x,y = extract_coords(df)

  # plot the complex numbers
  plot = px.scatter(x = x, y = y)
  plot["layout"].update(width = 1000, height = 1000, autosize = False)
  plot.show()

In [10]:
def generate_plot_pair(df1,df2):
  x1,y1 = extract_coords(df1)
  x2,y2 = extract_coords(df2)
  fig = make_subplots(rows=1, cols=2)
  fig.add_trace(
    go.Scatter(x = x1, y = y1,mode='markers'),
    
    row=1, col=1
  )
  fig.add_trace(
    go.Scatter(x = x2, y = y2,mode='markers'),
    row=1, col=2
  )
  fig.update_layout(height=1000, width=2000, title_text="Point comparison",autosize = False)
  fig.show()

In [12]:
obs_d_80 = X.iloc[2,].to_list()
obs_d_1200 = X.iloc[55,].to_list()

def transform_to_calc_affine(data):
    result = []
    for chunk in data.iterrows():
        for obs in chunk[1]:
            if obs.real < -2 and obs.imag > 2:
                result.append([obs.real, obs.imag])
        return np.array(result)


to_calc_80 = transform_to_calc_affine(X.iloc[:50,])
to_calc_1200 = transform_to_calc_affine(X.iloc[50:,])

transformation = cv2.getAffineTransform(to_calc_80, to_calc_1200)

error: ignored

In [13]:
for i in range(len(obs_d_80)):
    obs = obs_d_80[i]
    if obs.real < -2 and obs.imag > 2:
        arr = np.array([obs.real, obs.imag, 1])
        res = np.matmul(transformation[0], arr)
        obs_d_80[i] = complex(res[0], res[1])
        print(transformation[0], arr, res)
        aa = bb

NameError: ignored

In [14]:
# Example plot distance 80 
generate_plot(obs_d_80)

In [15]:
# Example plot distance 1200 
generate_plot(obs_d_1200)

In [16]:
# Example comparison attack 
generate_plot_pair(obs_d_80,obs_d_1200)

In [17]:
def modify_points(clust,data,dispersion = 1,rotation = 0,):
  # Dispersion: Mueve a los puntos x veces la distancia que tienen respecto al centro del cluster, en la misma direccion.
  for i,obs in data.iterrows():
    for j,sample in enumerate(obs):
      # Para detectar los puntos de los centroides que nos interesan, podemos hacer una aproximacion sencilla.
      # Tenemos un punto cercano al cluster.
      if round(sample.real) == clust.real and round(sample.imag) == clust.imag:
        dist = sample - clust
        data.iloc[i,j] = clust + dist*dispersion
  return data

In [18]:
clusters = [-3 + 3j,1+1j,-1-1j,1-3j]
data_prueba = X.iloc[:5,]

In [19]:
for clust in clusters: 
  data_prueba  = modify_points(clust,data_prueba,dispersion = 4)

In [20]:
generate_plot_pair(data_prueba.iloc[0,],obs_d_1200)

In [21]:
d_80_mod= X.iloc[:50,]

In [22]:
for clust in clusters: 
  d_80_mod  = modify_points(clust,d_80_mod,dispersion = 4)

In [23]:
# Parametros GMM
selCP=[1,7,10,15]
selCP_pos=[(-3,3),(1,1),(-1,-1),(1,-3)]
my_centers=[[-3,3],[-1,3],[1,3],[3,3],[-3,1],[-1,1],[1,1],[3,1],[-3,-1],[-1,-1],[1,-1],[3,-1], [-3,-3],[-1,-3],[1,-3],[3,-3]]

In [24]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

In [27]:
F=[]
for i in range(d_80.shape[0]):
    data=list(d_80.iloc[i,:])
    data=[[float(d.real), float(d.imag)] for d in data]
    gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data)
    mus=gmm.means_
    sigmas=gmm.covariances_
    obs = []
    for j in selCP_pos:
        mindist=None
        k_inc=None
        for k in range(16):
            d=L2dist(mus[k],j)
            if mindist is None or mindist>d:
                mindist=d
                k_inc=k
        covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
        #features = [*features, *mus[k_inc], *[covmat[0], covmat[3]]]
        obs = np.concatenate((obs, mus[k_inc]), axis=None)
        obs = np.concatenate((obs,covmat), axis=None)
    F.append(obs)

NameError: ignored

In [28]:
test = pd.DataFrame(columns = [['dist','mu_r_1','mu_i_1','sigma_rr_1','sigma_ri_1','sigma_ir_1','sigma_ii_1','mu_r_7','mu_i_7','sigma_rr_7','sigma_ri_7','sigma_ir_7','sigma_ii_7','mu_r_10','mu_i_10','sigma_rr_10','sigma_ri_10','sigma_ir_10','sigma_ii_10','mu_r_15','mu_i_15','sigma_rr_15','sigma_ri_15','sigma_ir_15','sigma_ii_15']])

In [29]:
F[0].tolist()

IndexError: ignored

In [30]:
aux = [None]
for i in range(len(F)):
  row = np.concatenate((aux, F[i]), axis=None)
  row = row.tolist()
  row = pd.Series(row, index = test.columns)
  test.append(row,ignore_index=True)